In [7]:
from urllib.request import urlretrieve
import os

## Define Data Paths

In [8]:
# Fill in your user name here....
username = "lpalum"
experiment_id = None

In [9]:
projectPath     = f"/home/jovyan/work/dsc402/{username}/mlmodels/profile/"
landingPath     = projectPath + "landing/"
silverDailyPath = projectPath + "daily/"
dimUserPath     = projectPath + "users/"
goldPath        = projectPath + "gold/"

if not os.path.exists('/home/jovyan/work/dsc402'):
    os.makedirs('/home/jovyan/work/dsc402')
if not os.path.exists('/home/jovyan/work/dsc402/{username}'):
    os.makedirs('/home/jovyan/work/dsc402/{username}')
if not os.path.exists('/home/jovyan/work/dsc402/{username}/mlmodels'):
    os.makedirs('/home/jovyan/work/dsc402/{username}/mlmodels')
if not os.path.exists(projectPath):
    os.makedirs(projectPath)
if not os.path.exists(landingPath):
    os.makedirs(landingPath)
if not os.path.exists(silverDailyPath):
    os.makedirs(silverDailyPath)
if not os.path.exists(dimUserPath):
    os.makedirs(dimUserPath)
if not os.path.exists(goldPath):
    os.makedirs(goldPath)

## Configure Database

In [6]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS dsc402_{username}")
spark.sql(f"USE dsc402_{username}");

NameError: name 'spark' is not defined

## Utility functions

In [ ]:
def retrieve_data(file: str) -> bool:
  """Download file from remote location to landing path."""
  base_url = "https://files.training.databricks.com/static/data/health-tracker/"
  urlretrieve(base_url + file, landingPath + file)
  return True

def load_delta_table(file: str, delta_table_path: str) -> bool:
  "Load a parquet file as a Delta table."
  parquet_df = spark.read.format("parquet").load(landingPath + file)
  parquet_df.write.format("delta").mode("overwrite").save(delta_table_path)
  return True

def process_file(file_name: str, path: str,  table_name: str) -> bool:
  """
  1. retrieve file
  2. load as delta table
  3. register table in the metastore
  """

  retrieve_data(file_name)
  print(f"Retrieve {file_name}.")

  load_delta_table(file_name, path)
  print(f"Load {file_name} to {path}")

  spark.sql(f"""
  DROP TABLE IF EXISTS {table_name}
  """)

  spark.sql(f"""
  CREATE TABLE {table_name}
  USING DELTA
  LOCATION "{path}"
  """)

  print(f"Register {table_name} using path: {path}")
